In [54]:
import pandas as pd
import numpy as np

from google_play_scraper import reviews, Sort

import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud

from konlpy.tag import Okt
from collections import Counter

from pprint import pprint

# Data

In [4]:
# df = pd.read_csv('./data/아이디어톤 설문지.csv')

In [55]:
# print(df.shape)
# df.head(3)

In [6]:
# df.columns = [
#     "timestamp",                # 타임스탬프
#     "gender",                   # 성별
#     "age",                      # 연령
#     "experience",               # 경력
#     "company_type",             # 재직 중인 기업의 형태를 선택해 주세요.
#     "is_it_industry",           # IT 기술 업계 종사자인가요?
#     "occupation",               # 직업
#     "current_roles",            # 현재 담당하고 계신 직무를 선택해주세요. (중복 선택 가능)
#     "issues_with_non_dev",      # 비개발 직군과 협업 시 주로 겪는 어려움을 선택해주세요. (중복 선택 가능)
#     "issues_with_dev",          # 개발 직군 간 협업 시 주로 겪는 어려움을 선택해주세요. (중복 선택 가능)
#     "collaboration_feedback",   # 협업과 소통 개선을 위한 의견을 자유롭게 작성해 주세요.
#     "use_service_intent",       # 타 개발 직군과의 원활한 소통을 위해 [하루 5분 IT 학습 서비스] 를 제공한다면 사용할 의향이 있으십니까?
#     "reason_for_intent",        # 해당 답변을 선택하신 이유는 무엇인가요?
#     "selected_roles",           # 현재 담당하고 계신 직무를 선택해주세요. (중복 선택 가능)
#     "issues_with_dev_comms",    # 개발자와의 소통/협업 시 주로 겪는 어려움을 선택해주세요. (중복 선택 가능)
#     "collaboration_ideas",      # 협업과 소통 개선을 위한 의견을 자유롭게 작성해 주세요.
#     "service_usage_intent",     # 개발자와의 원활한 소통을 위해 [하루 5분 IT 학습 서비스] 를 제공한다면 사용할 의향이 있으십니까?
#     "reason_for_usage_intent"   # 해당 답변을 선택하신 이유는 무엇인가요?
# ]

In [56]:
df = pd.read_csv("./data/form.csv")
print(df.shape)
df.head(3)

(49, 19)


,Unnamed: 0,timestamp,gender,age,experience,company_type,is_it_industry,occupation,current_roles,issues_with_non_dev,issues_with_dev,collaboration_feedback,use_service_intent,reason_for_intent,selected_roles,issues_with_dev_comms,collaboration_ideas,service_usage_intent,reason_for_usage_intent
0,0,2025/01/10 5:06:47 오후 GMT+9,남성,20대,1년 미만,스타트업,예,개발자 (데이터·인프라 직군 포함),Back-end 개발자;Front-end 개발자,개발 업무에 대한 이해 부족으로 소통이 어렵다.,기술적 용어 차이로 인해 소통이 어렵다.,NaN,예,도움될거같아서,NaN,NaN,NaN,NaN,NaN
1,1,2025/01/10 5:22:21 오후 GMT+9,남성,20대,1년~3년,중소기업,예,개발자 (데이터·인프라 직군 포함),Back-end 개발자,개발 업무에 대한 이해 부족으로 소통이 어렵다.;기술적 용어를 설명하는 데 어려움을...,업무 우선순위가 달라 갈등이 생긴다.;타 개발환경에 대한 이해도가 부족하다.,협업을 위해선 서로 개발 비개발 직군에 대한 이해도를 높이도록 이해해가며 노력해나가기,예,"타 개발 직군이 어떤 용어를 주로 사용하고, 환경을 어떻게 구성해가는지 이해할 수 ...",NaN,NaN,NaN,NaN,NaN
2,2,2025/01/10 5:23:07 오후 GMT+9,남성,20대,1년~3년,대기업,아니오,개발자 (데이터·인프라 직군 포함),시스템 엔지니어,요구사항이 불명확하여 업무가 지연된다.,기술적 용어 차이로 인해 소통이 어렵다.;업무 우선순위가 달라 갈등이 생긴다.,"듣는 사람의 수준을 고려한 의견 전달이 필요하고, 협업을 위해 타분야에 대한 이해도...",예,소프트웨어 분야의 트렌드를 익히고 배경 지식을 쌓아두는 역량은 모든 산업 분야에서 ...,NaN,NaN,NaN,NaN,NaN


# 분석

## 서비스 사용 희망 여부

### 직무별

In [40]:
expanded = (
    df.assign(current_roles=df["current_roles"].str.split(";"))
    .explode("current_roles")
)

# 직무별 use_service_intent 값 카운트
result = expanded.groupby(["current_roles", "use_service_intent"]).size().unstack(fill_value=0)

result['아니오 비율'] = round(result['아니오'] / (result['아니오'] + result['예']),2)
result['예 비율'] = round(result['예'] / (result['아니오'] + result['예']),2)

# 결과 확인
result

use_service_intent,아니오,예,아니오 비율,예 비율
current_roles,,,,
AI 엔지니어,2,3,0.40,0.60
Back-end 개발자,2,12,0.14,0.86
DevOps 엔지니어,2,1,0.67,0.33
Front-end 개발자,2,7,0.22,0.78
MLOps 엔지니어,2,1,0.67,0.33
게임 개발자,0,1,0.00,1.00
데이터 분석가,2,2,0.50,0.50
데이터 사이언티스트,0,1,0.00,1.00
데이터 엔지니어,0,4,0.00,1.00


### 기업규모별

In [42]:
round(df.groupby(['company_type'])['use_service_intent'].value_counts(normalize=True).to_frame(),2)

proportion
company_type use_service_intent            
대기업          예                         0.73
             아니오                       0.27
대학원          예                         1.00
스타트업         예                         0.62
             아니오                       0.38
중견기업         예                         0.73
             아니오                       0.27
중소기업         예                         1.00
프리랜서         아니오                       1.00

### IT 기업 여부

In [45]:
round(df.groupby(['is_it_industry'])['use_service_intent'].value_counts(normalize=True).to_frame(),2)

proportion
is_it_industry use_service_intent            
아니오            예                         0.80
               아니오                       0.20
예              예                         0.72
               아니오                       0.28

### 성별

In [46]:
round(df.groupby(['gender'])['use_service_intent'].value_counts(normalize=True).to_frame(),2)

proportion
gender use_service_intent            
남성     예                         0.77
       아니오                       0.23
여성     예                         0.67
       아니오                       0.33

### 연차별

In [53]:
round(df.groupby(['experience'])['use_service_intent'].value_counts(normalize=True).to_frame(), 2)

proportion
experience use_service_intent            
1년 미만      예                         0.87
           아니오                       0.13
1년~3년      예                         0.65
           아니오                       0.35
3년~5년      예                         0.67
           아니오                       0.33
5년 이상      아니오                       0.50
           예                         0.50